### Question 2 : Part 2

In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function, division
import string
import random
from data_utils import *
from rnn import *
import torch
import codecs
from tqdm import tqdm
import string
from math import log

#Set GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running using {device}")

Running using cuda


#Load vocabulary files

In [2]:
input_lang = torch.load('data-bin/fra.data')
output_lang = torch.load('data-bin/eng.data')

In [3]:
#Create and empty RNN model
encoder = EncoderRNN(input_size=input_lang.n_words, device=device)
attn_decoder = AttnDecoderRNN(output_size=output_lang.n_words, device=device)
#Load the saved model weights into the RNN model
encoder.load_state_dict(torch.load('model/encoder'))
attn_decoder.load_state_dict(torch.load('model/decoder'))

<All keys matched successfully>

In [4]:
#Return the decoder output given input sentence 
#Additionally, the previous predicted word and previous decoder state can also be given as input
def translate_single_word(encoder, decoder, sentence, decoder_input=None, decoder_hidden=None, max_length=MAX_LENGTH, device=device):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence, device)
        input_length = input_tensor.size()[0]
        
        encoder = encoder.to(device)
        decoder = decoder.to(device)
        
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        if decoder_input==None:
            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        else:
            decoder_input = torch.tensor([[output_lang.word2index[decoder_input]]], device=device) 
        
        if decoder_hidden == None:        
            decoder_hidden = encoder_hidden
        
        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
        return decoder_output.data, decoder_hidden

In [5]:
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		# select k best
		sequences = ordered[:k]
	return sequences

In [6]:
def beam_search(encoder,decoder,input_sentence,beam_size=1,max_length=MAX_LENGTH):
    decoded_output = []
    
    #Predicted the first word
    decoder_output, decoder_hidden = translate_single_word(encoder, decoder, input_sentence, decoder_input=None, decoder_hidden=None)
    
    #Get the probability of all output words
    decoder_output_probs = decoder_output.data
    idxs = beam_search_decoder(decoder_output_probs, beam_size) #get argMax for top three
    
	
    #Convert the predicted id to the word
    # first_word = output_lang.index2word[idx.item()]
    selected_first_words = [output_lang.index2word[idx[0]] for (idx, score) in idxs]
    
    #Add the predicted word to the output list and also set it as the previous prediction
    decoded_output.extend([[x] for x in selected_first_words]) # [[],[],[]]
    previous_decoded_output = selected_first_words # should be of length beam_size
    # print(f"selected words{selected_first_words}, and decoded_output {decoded_output}, previous_decoded_output {previous_decoded_output}")
    assert beam_size == len(selected_first_words)
    
    #Loop until the maximum length
    
    
    for i in range(max_length):
        beams = []
        for j, previous_decode in enumerate(previous_decoded_output):
            decoder_output, decoder_hidden = translate_single_word(encoder, decoder, input_sentence, previous_decode, decoder_hidden)
            decoder_output_probs = decoder_output.data
            beams.extend(decoder_output_probs)
        
        idxs = beam_search_decoder(beams, beam_size) 
        
        for idx in idxs:
            selected_word = output_lang.index2word[idx[0][0]]
            if selected_word == "EOS":
                break
            else:
                decoded_output[j].extend(selected_word)
                previous_decoded_output[j] = selected_word
                
    output_translation = " ".join(i for i in decoded_output[2])
    
    return output_translation

In [7]:
target_sentences = ["i can speak a bit of french .",
        "i ve bought some cheese and milk .",
        "boy where is your older brother ?",
        "i ve just started reading this book .",
        "she loves writing poems ."]

source_sentences = ["je parle un peu francais .",
            "j ai achete du fromage et du lait .",
            "garcon ou est ton grand frere ?",
            "je viens justement de commencer ce livre .",
            "elle adore ecrire des poemes ."]


def lines_from_file(file_name):
    lines = []
    with open(file_name) as f:
        lines = f.readlines()
    return lines
        

target = codecs.open('test_beam_1.out','w',encoding='utf-8')

beam_size = 3
for i,source_sentence in enumerate(source_sentences):

    target_sentence = normalizeString(target_sentences[i])
    input_sentence = normalizeString(source_sentence)
    
    hypothesis = beam_search(encoder, attn_decoder, input_sentence, beam_size=beam_size)
    print("S-"+str(i)+": "+input_sentence)
    print("T-"+str(i)+": "+target_sentence)
    print("H-"+str(i)+": "+hypothesis)
    print()
    target.write(hypothesis+'\n')
target.close()    



S-0: je parle un peu francais .
T-0: i can speak a bit of french .
H-0: you m m m a a f r e n c h . .

S-1: j ai achete du fromage et du lait .
T-1: i ve bought some cheese and milk .
H-1: we m a m m . . . . . . .

S-2: garcon ou est ton grand frere ?
T-2: boy where is your older brother ?
H-2: i s i s s t o o ? b r o t h e r ? ? ? ? ? ? ? ? ? ? ?

S-3: je viens justement de commencer ce livre .
T-3: i ve just started reading this book .
H-3: you m m a m t o t o t h e . . . . . . . . .

S-4: elle adore ecrire des poemes .
T-4: she loves writing poems .
H-4: they i s i s i s f r o m s o f . . . . . .



### Question 2: Part 3

In [8]:
# ! pip install sacrebleu

In [ ]:
from sacrebleu.metrics import BLEU
k_list = list(range(3,24))
belu_scores = []
def lines_from_file(file_name):
    lines = []
    with open(file_name) as f:
        lines = f.readlines()
    return lines
        
    
source_sentences = lines_from_file("data/valid.fra")
target_sentences = lines_from_file("data/test.eng")

for k in k_list:
    print("running for k ",k)
    beam_size = k
    hyps = []
    beam_size = k
    for i,source_sentence in enumerate(source_sentences):

        target_sentence = normalizeString(target_sentences[i])
        input_sentence = normalizeString(source_sentence)

        hypothesis = beam_search(encoder, attn_decoder, input_sentence, beam_size=beam_size)
        # print(hypothesis)
    hyps.append(hypothesis) 

    bleu = BLEU()
    result = bleu.corpus_score(hyps, source_sentences)
    print(result.score)
    belu_scores.append(result.score)

running for k  3
3.377156414337854
running for k  4


In [11]:
# from the plot best value for k is obtained
best_k = 4

In [ ]:
from matplotlib import pyplot as plt
plt.plot(k_list, belu_scores) 
plt.show()

### Question 2 : Part 4

In [12]:
def lines_from_file(file_name):
    lines = []
    with open(file_name) as f:
        lines = f.readlines()
    return lines
        
    
source_sentences = lines_from_file("data/test.fra")
target_sentences = lines_from_file("data/test.eng")

beam_size = best_k

for i,source_sentence in enumerate(source_sentences):

    target_sentence = normalizeString(target_sentences[i])
    input_sentence = normalizeString(source_sentence)

    hypothesis = beam_search(encoder, attn_decoder, input_sentence, beam_size=beam_size)
    print("S-"+str(i)+": "+input_sentence)
    print("T-"+str(i)+": "+target_sentence)
    print("H-"+str(i)+": "+hypothesis)
    print()

S-0: je t aime !
T-0: i love you !
H-0: he

S-1: fais tout ce qu il te dit .
T-1: do whatever he tells you .
H-1: i

S-2: c est difficile d avoir des idees geniales .
T-2: it s difficult to have great ideas .
H-2: she

S-3: je me suis etendu sur l herbe .
T-3: i laid myself on the grass .
H-3: he

S-4: je vais acheter une nouvelle voiture .
T-4: i am going to buy a new car .
H-4: you

S-5: je suis tres sensible a la chaleur .
T-5: i am very sensitive to heat .
H-5: they

S-6: mes parents m adorent .
T-6: i m loved by my parents .
H-6: they

S-7: j etais en mission a l etranger .
T-7: i was abroad on an assignment .
H-7: we

S-8: je m interesse a la natation .
T-8: i am interested in swimming .
H-8: he

S-9: j ai un ordinateur .
T-9: i have a computer .
H-9: he

S-10: je veux jouer de la guitare .
T-10: i want to play the guitar .
H-10: we

S-11: en ce qui me concerne je suis satisfait .
T-11: as for me i am satisfied .
H-11: you

S-12: dois je ouvrir la fenetre ?
T-12: do i have to ope